# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [21]:
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [22]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [23]:
import os
from glob import glob

temp = os.getenv("TEMP_DATA")
price_parquet_data_files = glob(os.path.join(temp, "parquet/*.parquet"))
price_parquet_data_files

['../../05_src/data/temp/parquet\\part.0.parquet',
 '../../05_src/data/temp/parquet\\part.1.parquet',
 '../../05_src/data/temp/parquet\\part.10.parquet',
 '../../05_src/data/temp/parquet\\part.11.parquet',
 '../../05_src/data/temp/parquet\\part.12.parquet',
 '../../05_src/data/temp/parquet\\part.13.parquet',
 '../../05_src/data/temp/parquet\\part.14.parquet',
 '../../05_src/data/temp/parquet\\part.15.parquet',
 '../../05_src/data/temp/parquet\\part.16.parquet',
 '../../05_src/data/temp/parquet\\part.17.parquet',
 '../../05_src/data/temp/parquet\\part.18.parquet',
 '../../05_src/data/temp/parquet\\part.19.parquet',
 '../../05_src/data/temp/parquet\\part.2.parquet',
 '../../05_src/data/temp/parquet\\part.20.parquet',
 '../../05_src/data/temp/parquet\\part.21.parquet',
 '../../05_src/data/temp/parquet\\part.22.parquet',
 '../../05_src/data/temp/parquet\\part.23.parquet',
 '../../05_src/data/temp/parquet\\part.24.parquet',
 '../../05_src/data/temp/parquet\\part.25.parquet',
 '../../05_src/

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [24]:
# Write your code below.
dd_px = dd.read_parquet(price_parquet_data_files).set_index("ticker")

dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']

)

dd_rets


C:\Users\chuch\AppData\Local\Temp\ipykernel_20252\3587515431.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
C:\Users\chuch\AppData\Local\Temp\ipykernel_20252\3587515431.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Close_lag_1,Returns,hi_lo_range
npartitions=59,,,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,float64,float64,float64
ALDX,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [25]:
# Write your code below.
freatures=dd_rets.compute()

freatures['ma_10_returns'] = freatures.groupby('ticker')['Returns'].rolling(10).mean().reset_index(level=0, drop=True)

freatures['ma_10_returns']

ticker
ACN          NaN
ACN          NaN
ACN          NaN
ACN          NaN
ACN          NaN
          ...   
ZIXI    0.014373
ZIXI    0.022675
ZIXI    0.017595
ZIXI    0.018008
ZIXI    0.011649
Name: ma_10_returns, Length: 239659, dtype: float64

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

I think it was not strictly necessary to convert to pandas, since Dask supports rolling operations using the .rolling() method. 

+ Would it have been better to do it in Dask? Why?

Using dask for the moving average would scale better for very large datasets since Dask processes data in chunks and supports better computation methods. If the dataset is small to fit in memory, pandas may be sufficient. For big data production pipelines, keeping calculations in Dask is usually preferable to avoid loading large data into memory.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.